In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
import nltk
from sklearn.model_selection import GridSearchCV
nltk.download('wordnet')
from sklearn.svm import SVC

[nltk_data] Downloading package wordnet to /Users/nathan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
col_names = ['marketplace','customer_id','review_id','product_id','product_parent','product_title','product_category','star_rating','helpful_votes','total_votes','vine','verified_purchase','review_headline','review_body','review_date']
cols = {}
for i in range(len(col_names)):
    print (str(i)+': '+col_names[i])
    cols[col_names[i]] = i 

0: marketplace
1: customer_id
2: review_id
3: product_id
4: product_parent
5: product_title
6: product_category
7: star_rating
8: helpful_votes
9: total_votes
10: vine
11: verified_purchase
12: review_headline
13: review_body
14: review_date


In [3]:
np.random.seed(500)

In [4]:
df = pd.read_csv('data/sample_02.csv')

df = df.sample(frac=.01, random_state=1)

In [5]:
helpful_percentage = []
for pos, total in zip(df['8'],df['9']):
    if total>0:
        helpful_percentage.append(float(pos)/float(total))
    else:
        helpful_percentage.append(0)
df['15'] = helpful_percentage

In [6]:
print(len(df))
df.head(3)

625


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
53304,US,49786746,R217S4F2FV9SRT,0965824233,173013649,The NAET Guide Book (4th Ed.),Books,5,13,29,N,N,The best Guide Book for a patient,It's a very good guide-book where you can get ...,11050,0.448276
21985,US,52661944,R295CEHQLEXS5G,0515120871,434295698,Finding the Dream (Dream Trilogy),Books,3,1,3,N,N,Sloppy writing,"This book, like all of Nora Roberts' books, is...",12200,0.333333
58996,US,52651244,RNAI0W4D8WWWL,0821761455,199563422,With All My Heart,Books,3,2,2,N,N,Not as good as the other two in the trilogy,I had been waiting patiently (sometimes unpati...,10650,1.000000


In [7]:
Corpus = pd.DataFrame()
Corpus['text'] = df['13']
lst = []
for x in df['15']:
    if x>=.66:
        lst.append('good')
    else:
        lst.append('bad')
Corpus['label'] = lst

In [8]:
len(Corpus['label'])

625

In [9]:
# Step - a : Remove blank rows if any.
Corpus['text'].dropna(inplace=True)
# Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
Corpus['text'] = [entry.lower() for entry in Corpus['text']]
# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
Corpus['text']= [word_tokenize(entry) for entry in Corpus['text']]
# Step - d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
lst = []
for index,entry in enumerate(Corpus['text']):
    
    if index%1000 ==0:
        print(index)
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    
    #Corpus.loc[index,'text_final'] = str(Final_words)
    lst.append(str(Final_words))
    
Corpus['text_final'] = lst

0


In [10]:
len(lst)

625

In [11]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['text_final'],Corpus['label'],test_size=0.3)




In [12]:
Train_Y

58394     bad
3298      bad
17466    good
23160    good
20783    good
         ... 
20213     bad
45770    good
14945    good
37614     bad
43690    good
Name: label, Length: 437, dtype: object

In [13]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

In [14]:
Tfidf_vect = TfidfVectorizer(max_features=10000)
Tfidf_vect.fit(Corpus['text_final'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [15]:
print(Tfidf_vect.vocabulary_)

{'good': 3393, 'get': 3333, 'quick': 6309, 'answer': 326, 'many': 4850, 'question': 6307, 'book': 873, 'like': 4626, 'nora': 5407, 'robert': 6751, 'enjoyable': 2613, 'character': 1227, 'interesting': 4138, 'relationship': 6519, 'michael': 5032, 'laura': 4521, 'daughter': 1914, 'really': 6407, 'lovely': 4738, 'however': 3818, 'writing': 8792, 'kinda': 4423, 'sloppy': 7257, 'quot': 6320, 'dare': 1895, 'dream': 2371, 'everyone': 2736, 'surprise': 7699, 'margo': 4868, 'josh': 4321, 'fell': 2965, 'love': 4735, 'seem': 6986, 'know': 4450, 'go': 3381, 'happen': 3576, 'explanation': 2823, 'find': 3012, 'coin': 1415, 'cliff': 1359, 'treasure': 8119, 'hide': 3714, 'cave': 1172, 'two': 8195, 'horse': 3802, 'name': 5260, 'lulu': 4757, 'tell': 7859, 'twice': 8190, 'byron': 1052, 'kate': 4378, 'expect': 2806, 'maybe': 4933, 'picky': 5871, 'point': 5971, 'confuse': 1567, 'follow': 3103, 'plot': 5950, 'inconsistency': 3990, 'detail': 2104, 'think': 7942, 'bear': 672, 'series': 7042, 'cordina': 1694, '

In [16]:
print(Train_X_Tfidf[3])

  (0, 8128)	0.32558888414963166
  (0, 7792)	0.24776030692642195
  (0, 7390)	0.3628726985894577
  (0, 7207)	0.2664954363786614
  (0, 5423)	0.23150086845259757
  (0, 5362)	0.24177757186422147
  (0, 4626)	0.11935788417984756
  (0, 3381)	0.14683477264127653
  (0, 3377)	0.34106306525831354
  (0, 3035)	0.2621900052323491
  (0, 2985)	0.2510212552699796
  (0, 2895)	0.3135861767968845
  (0, 1752)	0.3628726985894577
  (0, 873)	0.06294700096246933


In [17]:
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)

Naive Bayes Accuracy Score ->  58.51063829787234


In [18]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=6, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score ->  60.1063829787234


In [19]:
3994/len(Corpus)

6.3904

In [20]:
sum(Corpus['label']=='good')

393

## Grid Search SVM

In [21]:
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

scores = ['precision', 'recall']


In [22]:
clf = GridSearchCV(SVC(), tuned_parameters, cv=5,scoring='%s_macro' % scores[0])
clf.fit(Train_X_Tfidf,Train_Y)

/Users/nathan/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/nathan/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/nathan/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=None,
             param_grid=[{'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001],
                          'kernel': ['rbf']},
                         {'C': [1, 10, 100, 1000], 'kernel': ['linear']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='precision_macro', verbose=0)

In [23]:
print(clf.best_params_)

{'C': 1, 'kernel': 'linear'}
